In [1]:
!python preprocess_data.py -i data.csv -o dataset.data

Final dataset with size: | train (50400L, 10L) | test (12600L, 10L) | 


In [2]:
import numpy as np
import cPickle as cp

### Load the sensor data

In [3]:
def load_dataset(filename):

    f = file(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

In [4]:
print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('dataset.data')

Loading data...
 ..from file dataset.data
 ..reading instances: train (50400L, 10L), test (12600L, 10L)


In [5]:
first_conc_x = np.concatenate(X_train[0:15, :])
for i in range(15, X_train.shape[0], 15):
    first_conc_x = np.vstack((first_conc_x, np.concatenate(X_train[i:(i+15), :])))
X_train_kat = first_conc_x

In [6]:
first_conc_y = np.unique(y_train[0:15])[0]
for i in range(15, y_train.shape[0], 15):
    first_conc_y = np.vstack((first_conc_y, np.unique(y_train[i:(i+15)])[0]))
y_train_kat = first_conc_y

In [7]:
first_conc_x = np.concatenate(X_test[0:15, :])
for i in range(15, X_test.shape[0], 15):
    first_conc_x = np.vstack((first_conc_x, np.concatenate(X_test[i:(i+15), :])))
X_test_kat = first_conc_x

In [8]:
first_conc_y = np.unique(y_test[0:15])[0]
for i in range(15, y_test.shape[0], 15):
    first_conc_y = np.vstack((first_conc_y, np.unique(y_test[i:(i+15)])[0]))
y_test_kat = first_conc_y

In [9]:
y_train_kat.shape[0] == X_train_kat.shape[0]

True

In [10]:
y_test_kat.shape[0] == X_test_kat.shape[0]

True

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [12]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=200, alpha=0.0001,
                     solver='sgd', verbose=1, random_state=21, tol=0.000000001)

In [13]:
clf.fit(X_train_kat, y_train_kat.flatten())

Iteration 1, loss = 1.36945353
Iteration 2, loss = 1.16692584
Iteration 3, loss = 1.06977223
Iteration 4, loss = 0.98866270
Iteration 5, loss = 0.90971174
Iteration 6, loss = 0.83325455
Iteration 7, loss = 0.76313455
Iteration 8, loss = 0.70063571
Iteration 9, loss = 0.64533467
Iteration 10, loss = 0.59647714
Iteration 11, loss = 0.55358145
Iteration 12, loss = 0.51528813
Iteration 13, loss = 0.48151741
Iteration 14, loss = 0.45121496
Iteration 15, loss = 0.42501305
Iteration 16, loss = 0.40190031
Iteration 17, loss = 0.38048272
Iteration 18, loss = 0.36130584
Iteration 19, loss = 0.34452632
Iteration 20, loss = 0.32860593
Iteration 21, loss = 0.31456658
Iteration 22, loss = 0.30180586
Iteration 23, loss = 0.28975980
Iteration 24, loss = 0.27891054
Iteration 25, loss = 0.26867716
Iteration 26, loss = 0.25917997
Iteration 27, loss = 0.25040204
Iteration 28, loss = 0.24208351
Iteration 29, loss = 0.23469344
Iteration 30, loss = 0.22728886
Iteration 31, loss = 0.22058387
Iteration 32, los

C:\Users\DELL\Anaconda3\envs\py27\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=21, shuffle=True,
       solver='sgd', tol=1e-09, validation_fraction=0.1, verbose=1,
       warm_start=False)

In [14]:
y_pred_kat = clf.predict(X_test_kat)

In [15]:
accuracy_score(y_test_kat.flatten(), y_pred_kat)


0.9928571428571429

### Test Dataset

In [16]:
import os
import zipfile
import argparse
import numpy as np
import cPickle as cp
from io import BytesIO
from pandas import Series
import random

In [17]:
data = np.genfromtxt('testdata.csv', delimiter=',')

In [18]:
NB_SENSOR_CHANNELS = 10

NORM_MAX_THRESHOLDS = [200, 200, 200, 250000, 250000, 250000, 100, 100, 100, 100]

NORM_MIN_THRESHOLDS = [-200, -200, -200, -250000, -250000, -250000, -100, -100, -100, -100]

In [19]:
def normalize(data, max_list, min_list):
    max_list, min_list = np.array(max_list), np.array(min_list)
    diffs = max_list - min_list
    for i in np.arange(data.shape[1]):
        data[:, i] = (data[:, i]-min_list[i])/diffs[i]
    data[data > 1] = 0.99
    data[data < 0] = 0.00
    return data

In [20]:
def process_dataset_file(dataset):
    # Colums are segmentd into features and labels
    data_x = dataset

    # Perform linear interpolation
    data_x = np.array([Series(i).interpolate() for i in data_x.T]).T

    # Remaining missing data are converted to zero
    data_x[np.isnan(data_x)] = 0

    # All sensor channels are normalized
    data_x = normalize(data_x, NORM_MAX_THRESHOLDS, NORM_MIN_THRESHOLDS)

    return data_x

In [21]:
def generate_data(data):
    data_x = np.empty((0, 10))
    x = process_dataset_file(data)
    data_x = np.vstack((data_x, x))

    return data_x

In [22]:
test_data = generate_data(data)

In [23]:
first_conc_x = np.concatenate(test_data[0:15, :])
for i in range(15, test_data.shape[0], 15):
    first_conc_x = np.vstack((first_conc_x, np.concatenate(test_data[i:(i+15), :])))
test_data_kat = first_conc_x

In [24]:
clf.predict(test_data_kat).flatten()

array([2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 3,
       3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=uint8)